🔹 1. Nguyên lý hoạt động (không công thức nặng!)
🎯 Mục tiêu:
Dự đoán xác suất một mẫu thuộc về mỗi lớp, rồi chọn lớp có xác suất cao nhất.

🧠 Ý tưởng chính — dựa trên định lý Bayes:
"Xác suất một điều kiện xảy ra nếu biết một bằng chứng" 

Ví dụ:

“Nếu email có từ ‘miễn phí’, thì xác suất nó là spam là bao nhiêu?” 

🤔 Nhưng “Naive” (ngây thơ) ở đâu?
Thuật toán giả định các đặc trưng là ĐỘC LẬP với nhau.
Ví dụ: sự xuất hiện của từ “miễn phí” không ảnh hưởng đến từ “trúng thưởng”.
Trong thực tế, điều này rất hiếm khi đúng → nên gọi là “ngây thơ”.
Nhưng kỳ lạ thay: dù giả định sai, Naive Bayes vẫn hoạt động rất tốt!
💡 Cách nó “học”:
Không học trọng số như Logistic Regression.
Mà đếm tần suất trong dữ liệu huấn luyện:
Tỷ lệ email spam trong toàn bộ dữ liệu → xác suất prior
Tỷ lệ từ “miễn phí” xuất hiện trong email spam → khả năng xuất hiện từ đó nếu là spam
→ Khi có email mới, nó kết hợp các xác suất này để ra dự đoán.

✅ Ưu điểm:
Rất nhanh, ít tốn bộ nhớ.
Hiệu quả cao với dữ liệu văn bản (text classification).
Hoạt động tốt ngay cả khi dữ liệu huấn luyện ít.
❌ Hạn chế:
Giả định độc lập giữa các đặc trưng → có thể sai nặng nếu đặc trưng liên quan chặt (ví dụ: chiều cao & cân nặng).
Không cho ra mô hình phức tạp → khó bắt được mối quan hệ ẩn.

In [ ]:
import numpy as np

class SimpleGaussianNB:
    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        
        # Lưu thống kê cho mỗi lớp: mean và variance của từng đặc trưng
        self.mean = np.zeros((n_classes, n_features))
        self.var = np.zeros((n_classes, n_features))
        self.priors = np.zeros(n_classes)
        
        for idx, c in enumerate(self.classes):
            X_c = X[y == c]
            self.mean[idx, :] = X_c.mean(axis=0)
            self.var[idx, :] = X_c.var(axis=0)
            self.priors[idx] = X_c.shape[0] / float(n_samples)
    
    def _gaussian_probability(self, class_idx, x):
        """Tính xác suất x thuộc lớp class_idx theo phân phối chuẩn"""
        mean = self.mean[class_idx]
        var = self.var[class_idx]
        # Tránh chia cho 0
        var = np.clip(var, 1e-9, None)
        # Công thức phân phối chuẩn (không cần hằng số chuẩn hóa)
        numerator = np.exp(-0.5 * ((x - mean) ** 2) / var)
        denominator = np.sqrt(2 * np.pi * var)
        return np.prod(numerator / denominator)
    
    def predict(self, X):
        y_pred = []
        for x in X:
            posteriors = []
            for idx, c in enumerate(self.classes):
                # P(class) * P(x1|class) * P(x2|class) * ...
                prior = np.log(self.priors[idx])  # dùng log để tránh underflow
                likelihood = np.sum(np.log(self._gaussian_probability(idx, x) + 1e-9))
                posterior = prior + likelihood
                posteriors.append(posterior)
            # Chọn lớp có posterior lớn nhất
            y_pred.append(self.classes[np.argmax(posteriors)])
        return np.array(y_pred)

# --- Thử nghiệm ---
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Dùng Iris (đặc trưng là số thực → phù hợp Gaussian NB)
iris = load_iris()
X, y = iris.data, iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Huấn luyện mô hình tự code
nb_model = SimpleGaussianNB()
nb_model.fit(X_train, y_train)
y_pred = nb_model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"✅ Độ chính xác (Naive Bayes tự code): {acc:.2%}")

# So sánh với sklearn
from sklearn.naive_bayes import GaussianNB
sk_nb = GaussianNB()
sk_nb.fit(X_train, y_train)
sk_acc = sk_nb.score(X_test, y_test)
print(f"🔍 Sklearn GaussianNB độ chính xác: {sk_acc:.2%}")

 3. Khi nào dùng Naive Bayes?
Phân loại văn bản
: spam detection, phân tích cảm xúc, phân loại chủ đề
Dữ liệu có
mối quan hệ mạnh giữa các đặc trưng
(ví dụ: ảnh pixel lân cận)
Dữ liệu huấn luyện ít
nhưng số đặc trưng lớn (text có hàng nghìn từ)
Bạn cần
ước lượng xác suất chính xác tuyệt đối
(NB thường cho xác suất bị lệch)
Cần
mô hình cực nhanh
để huấn luyện & dự đoán
Đặc trưng
không tuân theo phân phối chuẩn
(với Gaussian NB)
Làm
baseline đơn giản
cho bài toán phân loại
Bài toán yêu cầu
giải thích chi tiết mối quan hệ giữa đặc trưng